In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/Thesis/Data/compressed_data.zip
!unzip /content/drive/MyDrive/Thesis/Data/demand_graphs.pkl-005.zip

Archive:  /content/drive/MyDrive/Thesis/Data/compressed_data.zip
  inflating: demand_graphs.pkl.npz   
  inflating: final_model_input_partial_scale_4.csv  
Archive:  /content/drive/MyDrive/Thesis/Data/demand_graphs.pkl-005.zip
  inflating: demand_graphs.pkl-005.npz  


In [3]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.7 MB/s eta 0:00:00


In [4]:
import os
import math
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch_geometric.nn import SAGEConv, global_mean_pool
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, random_split
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse

In [5]:
directory = '/content/drive/MyDrive/Thesis'
data_dir = directory + "/Data"
models_dir = directory + "/models"

In [7]:
timestamps = np.load(f'{data_dir}/demand_graph_timestamps.pkl.npz')
stamps = [timestamps[f'arr_{i}'] for i in range(4379, len(timestamps))] #17516
datetimes = [datetime.datetime(int(arr[0][0]), int(arr[0][1]), int(arr[0][2]), int(arr[0][3])) for arr in stamps]

In [8]:
datetimes[-1]

datetime.datetime(2024, 3, 31, 23, 0)

In [9]:
datetimes[0]

datetime.datetime(2022, 7, 2, 13, 0)

In [10]:
station_clusters = [
    74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,
    87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99,
    100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
    113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
    126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
    139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151,
    152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164,
    165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177,
    178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190,
    191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203,
    205, 206, 208, 209, 210, 211, 212, 213, 214, 216, 217, 218, 219,
    220, 222, 223, 224, 225, 226, 227, 228, 230, 231, 232, 233, 234,
    235, 237, 238, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249,
    250, 251, 252, 253, 255, 257, 258, 261, 264, 265, 266, 274, 275,
    278
]

num_stations = len(station_clusters)

In [11]:
df = pd.read_csv(f'final_model_input_partial_scale_4.csv')[[
    'start_station_cluster',
    'started_at_hourly',
    'demand'
]]


# Rename the column just once
df = df.rename(columns={"started_at_hourly": "datetime"})
df['datetime'] = pd.to_datetime(df['datetime'])
# Create a DataFrame with all combinations of datetimes and station_clusters
all_combinations = pd.DataFrame(
    [(dt, sc) for dt in datetimes for sc in station_clusters],
    columns=["datetime", "start_station_cluster"]
)

weather_data = pd.read_csv(f"{data_dir}/weather_data.csv")
weather_data['datetime'] = pd.to_datetime(weather_data['started_at_hourly'])

weather_data = weather_data[[
    'datetime',
    'temp',
    'dwpt',
    'rhum',
    'prcp',
    'wdir',
    'wspd',
    'pres'
]]

# Merge with the original DataFrame to include 'demand' where it exists
output_df = all_combinations.merge(
    df[['demand', 'start_station_cluster', 'datetime']],
    on=["datetime", "start_station_cluster"],
    how="left"
)

# Fill missing values with 0
output_df['demand'] = output_df['demand'].fillna(0)

weather_data = weather_data[weather_data['datetime'] >= datetimes[0]]
output_df = output_df[output_df['datetime'] >= datetimes[0]]
target_array = output_df['demand'].values



In [12]:
del df
del all_combinations
del timestamps
del output_df

In [13]:
datetimes[-1]

datetime.datetime(2024, 3, 31, 23, 0)

In [14]:
target = target_array.reshape(-1, num_stations)

In [15]:
target.max()

np.float64(355.0)

In [16]:
datetimes[4379]

datetime.datetime(2023, 1, 1, 0, 0)

In [17]:
stamps[4379][0]

array([2.023e+03, 1.000e+00, 1.000e+00, 0.000e+00, 1.000e+00])

In [18]:
target.shape

(15321, 183)

In [19]:
loaded = np.load('demand_graphs.pkl-005.npz')
demand_graphs = [loaded[f'arr_{i}'] for i in range(4376, 17515)] #4378

In [20]:
adj_matrices = torch.tensor(demand_graphs)

# Number of stations
num_stations = adj_matrices.shape[1]

<ipython-input-20-4adbaaf13251>:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  adj_matrices = torch.tensor(demand_graphs)


In [21]:
del demand_graphs

In [22]:
len(target)

15321

In [23]:
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse

# Convert adjacency matrices to PyTorch Geometric format for each hour
def create_datalist(adj_matrices, target, datetimes, weather, target_index=0):
  data_list = []
  for i in range(adj_matrices.shape[0]):  # Iterate over each hour
      adj_matrix = adj_matrices[i]
      y = target[i+target_index]
      datetime_features = datetimes[i+target_index][0]
      weather_features = torch.tensor(weather_data[[
          'dwpt',
          'rhum',
          'prcp',
          'wdir',
          'wspd',
          'pres']].iloc[i+target_index].to_numpy(), dtype=torch.float32)
      # Convert dense adjacency matrix to edge_index (sparse representation)
      edge_index, edge_attr = dense_to_sparse(adj_matrix)
      # Use an identity matrix for node features (can be replaced with other features)
      x = torch.eye(num_stations)
      # Create a Data object
      y = torch.tensor(y, dtype=torch.float32)
      datetime_features = torch.tensor(datetime_features, dtype=torch.float32)
      data = Data(
          x=adj_matrix.float(),
          y=y,
          edge_index=edge_index,
          edge_attr=edge_attr,
          datetime_features=datetime_features,
          weather_features=weather_features
        )
      data_list.append(data)

  print(data_list[0])

  for data in data_list:
      data.edge_attr = data.edge_attr.float()

  return data_list
data_list = create_datalist(adj_matrices, target, stamps, weather_data)

Data(x=[183, 183], edge_index=[2, 195], edge_attr=[195], y=[183], datetime_features=[5], weather_features=[6])


In [24]:
weather_len = len(['dwpt', 'rhum', 'prcp', 'wdir', 'wspd', 'pres'])

In [25]:
#del target
del adj_matrices
del target_array

In [26]:
def create_lagged_dataset(graph_data_list, lag=2):
    lagged_dataset = []
    for i in range(lag, len(graph_data_list)):
        current_graph = graph_data_list[i]
        lag_graphs = graph_data_list[i - lag:i]  # Collect lag graphs
        lagged_dataset.append((current_graph, lag_graphs))
    return lagged_dataset

In [27]:
lagged_dataset = create_lagged_dataset(data_list)

In [28]:
len(lagged_dataset)

13137

In [29]:
len(data_list)

13139

In [30]:
#data_list[360].to_dict()['edge_attr']

In [31]:

import torch.nn.functional as F

class GNNLayer(torch.nn.Module):
    def __init__(self, in_channels, h1, out_channels, datetime_feats_len, weather_feats_len, num_layers, dropout_prob):
        super(GNNLayer, self).__init__()
        self.convs = nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, h1))

        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(h1, h1))
        self.leaky_relu = nn.LeakyReLU()
        self.dropout_prob = dropout_prob
        self.datetime_feats_len = datetime_feats_len
        self.weather_feats_len = weather_feats_len

        self.convs.append(SAGEConv(h1, out_channels))

    def forward(self, x, edge_index, batch, datetime_features, weather_features):
      for conv in self.convs[:-1]:
        x = self.leaky_relu(conv(x, edge_index))
        x = F.dropout(x, p=self.dropout_prob, training=self.training)
      x = self.convs[-1](x, edge_index)

      x = global_mean_pool(x, batch)
      datetime_feats = datetime_features[0:self.datetime_feats_len]
      datetime_feats = datetime_feats.unsqueeze(0).expand(x.shape[0], -1)

      weather_feats = weather_features[0:self.weather_feats_len]
      weather_feats = weather_feats.unsqueeze(0).expand(x.shape[0], -1)

      combined_features = torch.cat([x, datetime_feats], dim=1)
      combined_features = torch.cat([combined_features, weather_feats], dim=1)
      return combined_features

In [32]:


class GNNForDemandPrediction(torch.nn.Module):
    def __init__(
          self, in_channels, out_channels=50, datetime_feats_len=5, weather_feats_len=6, lag=3, h1=100, num_layers=2, num_layers_gru=1, fc_hidden_dim = 256, gru_hidden_dim=128, dropout_prob=0.2
        ):
        super(GNNForDemandPrediction, self).__init__()

        self.gnn = GNNLayer(in_channels, h1, out_channels, datetime_feats_len, weather_feats_len, num_layers, dropout_prob)

        self.gru = nn.GRU(
            (out_channels+datetime_feats_len+weather_feats_len)*lag,
            gru_hidden_dim,
            num_layers_gru,
            batch_first=True,
            bidirectional=True,
            dropout=dropout_prob
        )
        self.fc1 = torch.nn.Linear(gru_hidden_dim, fc_hidden_dim)
        self.fc2 = torch.nn.Linear(fc_hidden_dim, in_channels)
        self.dropout_prob = dropout_prob
        self.leaky_relu = nn.LeakyReLU()

    #x, edge_index, batch, datetime_features, weather_features
    def forward(self, batch):
        current_graphs, lag_graphs = batch

        current_graphs.to(device)
        current_x = torch.cat([
            self.gnn(g.x, g.edge_index, g.batch, g.datetime_features, g.weather_features) for g in [current_graphs]
        ], dim=0)
        lag_x_list = []
        for i in lag_graphs:
          #print(i)
          i.to(device)
          x, edge_index, batch_indices, datetime_features, weather_features =\
              i.x, i.edge_index, i.batch, i.datetime_features, i.weather_features
          lag_x = torch.cat([self.gnn(x, edge_index, batch_indices, datetime_features, weather_features)], dim=1)
          #gnn_embedding = gnn_embedding.unsqueeze(1)
          lag_x_list.append(lag_x)
        lag_x = torch.stack(lag_x_list)
        #gnn_embedding = lag_x.mean(dim=0)
        gnn_embedding = current_x
        for i in lag_x:
          gnn_embedding = torch.cat([gnn_embedding, i], dim=1)
        gnn_embedding = gnn_embedding.unsqueeze(1)
        output, hidden = self.gru(gnn_embedding)

        x = self.leaky_relu(self.fc1(hidden[-1]))

        x = F.dropout(x, p=self.dropout_prob, training=self.training)

        predicted_demand = self.fc2(x)

        return predicted_demand



In [33]:
stamps[0][0].shape[0]

5

In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [35]:
# Instantiate the model
model = GNNForDemandPrediction(
    in_channels=num_stations,
    h1=100,
    datetime_feats_len = stamps[0][0].shape[0],
    weather_feats_len=weather_len,
    num_layers_gru=20,
    num_layers=20
  ).to(device)

# Define the loss function (MSE for regression) and the optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)

In [36]:
model

GNNForDemandPrediction(
  (gnn): GNNLayer(
    (convs): ModuleList(
      (0): SAGEConv(183, 100, aggr=mean)
      (1-18): 18 x SAGEConv(100, 100, aggr=mean)
      (19): SAGEConv(100, 50, aggr=mean)
    )
    (leaky_relu): LeakyReLU(negative_slope=0.01)
  )
  (gru): GRU(183, 128, num_layers=20, batch_first=True, dropout=0.2, bidirectional=True)
  (fc1): Linear(in_features=128, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=183, bias=True)
  (leaky_relu): LeakyReLU(negative_slope=0.01)
)

In [37]:
num_stations

183

In [38]:

#dataloader = DataLoader(data_list, batch_size=32, shuffle=True)
dataloader = DataLoader(lagged_dataset, batch_size=32, shuffle=True)

In [39]:
data_list[0]

Data(x=[183, 183], edge_index=[2, 195], edge_attr=[195], y=[183], datetime_features=[5], weather_features=[6])

In [40]:
# Training function
def train(model, data_list, optimizer, criterion, epochs=20):
    model.train()  # Set the model to training mode
    for epoch in range(epochs):
        total_loss = 0  # Keep track of total loss

        # Loop over the data (for each hour)
        for batch in dataloader:
            optimizer.zero_grad()  # Clear the gradients
            # Forward pass: predict demand for this hour
            #x, edge_index, batch_indices = batch.x, batch.edge_index, batch.batch  # Edge indices
            predicted_demand = model(batch)#, batch.datetime_features
            # Calculate loss (difference between predicted and actual demand)
            loss = criterion(predicted_demand.reshape(-1), batch[0].y.to(device))

            # Backpropagation and optimization step
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {total_loss / len(data_list)}')

# Train the model using data from all hours
train(model, data_list, optimizer, criterion, epochs=30)

Epoch 0, Loss: 0.12119917008909281
Epoch 10, Loss: 0.11932316370973559
Epoch 20, Loss: 0.11905517979330174


In [41]:
torch.Size([5856, 50])
torch.Size([32, 55])
torch.Size([32, 55])
torch.Size([128])
torch.Size([183])

torch.Size([183])

In [42]:
f"{datetime.datetime.now()}"

'2025-03-30 20:21:44.747010'

In [43]:
model_name = f'{models_dir}/gnn_sage_weather_datetime_2023_{datetime.datetime.now()}.pth'

In [44]:
model_name

'/content/drive/MyDrive/Thesis/models/gnn_sage_weather_datetime_2023_2025-03-30 20:21:44.752334.pth'

In [45]:
torch.save(model.state_dict(), model_name)



In [46]:
del data_list

In [47]:
#model_name = f'{models_dir}/gnn_gat_weather_datetime_2023_2024-12-28 15:54:07.159004.pth'

In [48]:
# Instantiate the model
model = GNNForDemandPrediction(
    in_channels=num_stations,
    h1=100,
    datetime_feats_len = stamps[0][0].shape[0],
    weather_feats_len=weather_len,
    num_layers_gru=20,
    num_layers=20
  ).to(device)

model.load_state_dict(torch.load(model_name))

<All keys matched successfully>

In [49]:
model

GNNForDemandPrediction(
  (gnn): GNNLayer(
    (convs): ModuleList(
      (0): SAGEConv(183, 100, aggr=mean)
      (1-18): 18 x SAGEConv(100, 100, aggr=mean)
      (19): SAGEConv(100, 50, aggr=mean)
    )
    (leaky_relu): LeakyReLU(negative_slope=0.01)
  )
  (gru): GRU(183, 128, num_layers=20, batch_first=True, dropout=0.2, bidirectional=True)
  (fc1): Linear(in_features=128, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=183, bias=True)
  (leaky_relu): LeakyReLU(negative_slope=0.01)
)

In [50]:
loaded = np.load('demand_graphs.pkl-005.npz')
demand_graphs = [loaded[f'arr_{i}'] for i in range(17513, len(loaded)-1)] #17515

In [51]:
datetimes[13138]

datetime.datetime(2024, 1, 1, 0, 0)

In [52]:
len(demand_graphs)

2186

In [53]:
15321-13137

2184

In [54]:
adj_matrices = torch.tensor(demand_graphs)
test_data_list = create_datalist(adj_matrices, target, stamps, weather_data, target_index=13135)

Data(x=[183, 183], edge_index=[2, 113], edge_attr=[113], y=[183], datetime_features=[5], weather_features=[6])


In [55]:
lagged_dataset = create_lagged_dataset(test_data_list)

In [56]:
len(lagged_dataset)

2184

In [57]:
del demand_graphs
del adj_matrices

In [58]:
test_dataloader = DataLoader(lagged_dataset, batch_size=1, shuffle=True)

In [59]:

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
criterion = torch.nn.MSELoss()
# Testing the model (evaluation mode)
def test(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0

    preds = []
    actual = []
    with torch.no_grad():  # No need to track gradients during testing
        for batch in dataloader:
            predicted_demand = model(batch)  # Forward pass
            #print(predicted_demand.reshape(-1).shape)

            preds.extend(predicted_demand.cpu().reshape(-1))
            #print(batch.y.reshape(-1).shape)
            actual.extend(batch[0].y.cpu().reshape(-1))
            #loss = criterion(predicted_demand, data.y)  # Compute loss
            #total_loss += loss.item()
    df = pd.DataFrame({"actual": actual, "pred": preds})
    non_zero = df.query('actual != 0')
    zeros = df.query('actual == 0')


    mse_score = mean_squared_error(df['actual'], df['pred'])
    rmse_score = np.sqrt(mse_score)
    mae_score = mean_absolute_error(df['actual'], df['pred'])
    mape_score = mean_absolute_percentage_error(df['actual']+1, df['pred']+1)
    print("overall")
    print("MSE:", mse_score)
    print("RMSE:", rmse_score)
    print("MAE:", mae_score)
    print("MAPE:", mape_score)


    mse_score = mean_squared_error(non_zero['actual'], non_zero['pred'])
    rmse_score = np.sqrt(mse_score)
    mae_score = mean_absolute_error(non_zero['actual'], non_zero['pred'])
    mape_score = mean_absolute_percentage_error(non_zero['actual']+1, non_zero['pred']+1)
    print()
    print("Non-zero")
    print("MSE:", mse_score)
    print("RMSE:", rmse_score)
    print("MAE:", mae_score)
    print("MAPE:", mape_score)

    mse_score = mean_squared_error(zeros['actual'], zeros['pred'])
    rmse_score = np.sqrt(mse_score)
    mae_score = mean_absolute_error(zeros['actual'], zeros['pred'])
    mape_score = mean_absolute_percentage_error(zeros['actual']+1, zeros['pred']+1)
    print()
    print("Zeros")
    print("MSE:", mse_score)
    print("RMSE:", rmse_score)
    print("MAE:", mae_score)
    print("MAPE:", mape_score)
    #print(f'Test Loss: {total_loss / len(data_list)}')

# Test the model
test(model, test_dataloader)

overall
MSE: 5.703620526566604
RMSE: 2.3882253927480552
MAE: 0.7654953730675859
MAPE: 0.2712691274126156

Non-zero
MSE: 25.804866448171104
RMSE: 5.07984905761688
MAE: 2.8078767735206465
MAPE: 0.4538541536138454

Zeros
MSE: 0.3618726649631895
RMSE: 0.601558529956304
MAE: 0.2227485940371903
MAPE: 0.22274859405488173


In [60]:
2.325295031065866

2.325295031065866

In [ ]:

5.703620526566604
2.3882253927480552
0.7654953730675859
0.2712691274126156

25.804866448171104
5.07984905761688
2.8078767735206465
0.4538541536138454

0.3618726649631895
0.601558529956304
0.2227485940371903
0.22274859405488173

In [61]:
#from psutil import *
## This code will return the number of CPU
#print("Number of CPU: ", cpu_count())
## This code will return the CPU info
#!nvidia-smi